In [1]:
import catboost
from catboost import CatBoostClassifier, Pool

from gs_ac_gan_training import prepare_test_data
from utils.util import *

output_folder = "classifier_analysis"
hapt_genotypes_path = '../resource/train_0.8_sub_pop.csv'
experiment_results = '../experiment_results/discriminator_0.8_test'
target_column = 'Population code'
(x_train, y_train), _, _, _ = init_dataset(hapt_genotypes_path=hapt_genotypes_path, target_column=target_column, without_extra_data=True)
test_dataset = prepare_test_data(experiment_results, test_path="../resource/test_0.2_sub_pop.csv")
y_train = np.argmax(y_train, axis=1)
print(y_train)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/Users/shakedahronoviz/code/GS-AC-GAN/utils/util.py:614: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_to_pca.loc[:, "pca_sum"] = np.sqrt(np.square(X_pca).sum(axis=1))


Metal device set to: Apple M1 Pro
[17 17 17 ... 14 14 14]


2023-04-16 17:33:25.541152: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-16 17:33:25.541194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# Create a CatBoost classifier
clf = CatBoostClassifier(iterations=1000, learning_rate=0.1)
train_pool = catboost.Pool(data=x_train, label=y_train)
# Train the classifier
clf.fit(train_pool)

0:	learn: 3.1250885	total: 2.51s	remaining: 41m 49s
1:	learn: 3.0075206	total: 4.25s	remaining: 35m 19s
2:	learn: 2.9280272	total: 6.06s	remaining: 33m 34s
3:	learn: 2.8511907	total: 8.09s	remaining: 33m 33s
4:	learn: 2.7791662	total: 9.75s	remaining: 32m 19s
5:	learn: 2.7272418	total: 11.4s	remaining: 31m 35s
6:	learn: 2.6770835	total: 13.2s	remaining: 31m 14s
7:	learn: 2.6414199	total: 14.8s	remaining: 30m 37s
8:	learn: 2.5919872	total: 16.6s	remaining: 30m 23s
9:	learn: 2.5488188	total: 18.7s	remaining: 30m 52s
10:	learn: 2.5160406	total: 20.6s	remaining: 30m 51s
11:	learn: 2.4818268	total: 22.7s	remaining: 31m 5s
12:	learn: 2.4481215	total: 24.5s	remaining: 31m 3s
13:	learn: 2.4227151	total: 26.5s	remaining: 31m 7s
14:	learn: 2.3941021	total: 28.5s	remaining: 31m 12s
15:	learn: 2.3730441	total: 30.4s	remaining: 31m 10s
16:	learn: 2.3521969	total: 32.3s	remaining: 31m 10s
17:	learn: 2.3311420	total: 34.4s	remaining: 31m 17s
18:	learn: 2.3096581	total: 36.4s	remaining: 31m 17s
19:	le

In [3]:
def evaluate_and_save_catboost_model(test_dataset):
    shuffled_dataset = tensorflow.data.Dataset.from_tensor_slices(test_dataset).shuffle(
        test_dataset[0].shape[0]).batch(test_dataset[0].shape[0], drop_remainder=True)
    for x_batch_real, y_batch_real in shuffled_dataset:
        test_pool = Pool(np.array(x_batch_real))
        y_pred = clf.predict(test_pool)
        pd.DataFrame({"class_pred": y_pred.flatten(), "class_real": np.array(y_batch_real)}).to_csv(
            os.path.join(output_folder, "catboost_results.csv"), index=False)


In [4]:
evaluate_and_save_catboost_model(test_dataset)